### Обучение пайплайна

1. Загрузим датасет https://www.kaggle.com/blackmoon/russian-language-toxic-comments
2. Подготовим пайплайн с  препроцессингом (tfidf) на текстовых данных
3. Обучим модель и сохраним на диск предобученный пайплайн

In [1]:
import pandas as pd
import dill
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from sklearn.metrics import f1_score
from sklearn.ensemble import GradientBoostingClassifier

#working with text
from sklearn.feature_extraction.text import TfidfVectorizer

#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score

#imputer
from sklearn.impute import SimpleImputer

/Users/ekaterinarazdumina/.conda/envs/env/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


Загрузим данные

In [2]:
df = pd.read_csv("./data.csv")
df.head(3)

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0


Разделим данные на train/test и сохраним тестовую выборку на диск

In [3]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('toxic', 1), 
                                                    df['toxic'], test_size=0.33, random_state=42)
#save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)
#save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [4]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class TextImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X[self.key] = X[self.key].fillna(self.value)
        return X

In [5]:
#combine
comment = Pipeline([
                ('imputer', TextImputer('comment', '')),
                ('selector', ColumnSelector(key='comment')),
                ('tfidf', TfidfVectorizer(max_df=0.9, min_df=10))
            ])

feats = FeatureUnion([('comment', comment),
                      ])

In [6]:
%%time

pipeline = Pipeline([
    ('features', feats),
    ('classifier', GradientBoostingClassifier (random_state=42)),
])

pipeline.fit(X_train, y_train)

CPU times: user 3.21 s, sys: 29 ms, total: 3.24 s
Wall time: 3.25 s


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('comment',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='comment',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='comment')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer(max_df=0.9,
                                                                                  min_df=10))]))])),
                ('classifier', GradientBoostingClassifier(random_state=42))])

Посмотрим, как выглядит наш pipeline и замерим метрики 

In [7]:
pipeline.steps

[('features',
  FeatureUnion(transformer_list=[('comment',
                                  Pipeline(steps=[('imputer',
                                                   TextImputer(key='comment',
                                                               value='')),
                                                  ('selector',
                                                   ColumnSelector(key='comment')),
                                                  ('tfidf',
                                                   TfidfVectorizer(max_df=0.9,
                                                                   min_df=10))]))])),
 ('classifier', GradientBoostingClassifier(random_state=42))]

In [8]:
predictions = pipeline.predict_proba(X_test.iloc[:500])

<ipython-input-4-803d89350675>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[self.key] = X[self.key].fillna(self.value)


In [9]:
roc_auc_score(y_score=predictions[:, 1][:], y_true=y_test.iloc[:500])

0.8213747848537005

Сохраним модель, обученную на train выборке, чтобы замерить  (пайплайн)

In [10]:
with open("pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)